# Conditional Real NVP

In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from tensorboardX import SummaryWriter

from tqdm import tqdm

batch_size = 128
epochs = 100
seed = 1
torch.manual_seed(seed)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [2]:
root = '../data'
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambd=lambda x: x.view(-1))])
kwargs = {'batch_size': batch_size, 'num_workers': 1, 'pin_memory': True}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=root, train=True, transform=transform, download=True),
    shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=root, train=False, transform=transform),
    shuffle=False, **kwargs)

In [3]:
from pixyz.distributions import Normal, InverseTransformedDistribution
from pixyz.flows import AffineCouplingLayer, FlowList, BatchNorm1d, ShuffleLayer, PreprocessLayer, ReverseLayer
from pixyz.models import ML

In [4]:
x_dim = 28*28
y_dim = 10
z_dim = x_dim

In [5]:
# prior model p(z)
loc = torch.tensor(0.).to(device)
scale = torch.tensor(1.).to(device)
prior = Normal(loc=loc, scale=scale, var=["z"], dim=z_dim, name="p_prior")

In [6]:
class ScaleTranslateNet(nn.Module):
    def __init__(self, in_features, hidden_features):
        super().__init__()
        self.fc1 = nn.Linear(in_features+y_dim, hidden_features)
        self.fc2 = nn.Linear(hidden_features, hidden_features)
        self.fc3_s = nn.Linear(hidden_features, in_features)
        self.fc3_t = nn.Linear(hidden_features, in_features)

    def forward(self, x, y):
        hidden = F.relu(self.fc2(F.relu(self.fc1(torch.cat([x, y], 1)))))
        log_s = torch.tanh(self.fc3_s(hidden))
        t = self.fc3_t(hidden)
        return log_s, t

In [7]:
# flow
flow_list = []
num_block = 5

flow_list.append(PreprocessLayer())

for i in range(num_block):
        flow_list.append(AffineCouplingLayer(in_channels=x_dim,
                                             scale_translate_net=ScaleTranslateNet(x_dim, 1028),
                                             inverse_mask=(i%2!=0)))
        
        flow_list.append(BatchNorm1d(x_dim))
        
f = FlowList(flow_list)

In [8]:
# inverse transformed distribution (z -> f^-1 -> x)
p = InverseTransformedDistribution(prior=prior, flow=f, var=["x"], cond_var=["y"])
p.to(device)

InverseTransformedDistribution(
  (prior): Normal()
  (flow): FlowList(
    (0): PreprocessLayer()
    (1): AffineCouplingLayer(
      in_features=784, mask_type=channel_wise, inverse_mask=False
      (scale_translate_net): ScaleTranslateNet(
        (fc1): Linear(in_features=794, out_features=1028, bias=True)
        (fc2): Linear(in_features=1028, out_features=1028, bias=True)
        (fc3_s): Linear(in_features=1028, out_features=784, bias=True)
        (fc3_t): Linear(in_features=1028, out_features=784, bias=True)
      )
    )
    (2): BatchNorm1d()
    (3): AffineCouplingLayer(
      in_features=784, mask_type=channel_wise, inverse_mask=True
      (scale_translate_net): ScaleTranslateNet(
        (fc1): Linear(in_features=794, out_features=1028, bias=True)
        (fc2): Linear(in_features=1028, out_features=1028, bias=True)
        (fc3_s): Linear(in_features=1028, out_features=784, bias=True)
        (fc3_t): Linear(in_features=1028, out_features=784, bias=True)
      )
    )
 

In [9]:
model = ML(p, optimizer=optim.Adam, optimizer_params={"lr":1e-3})
print(model)

Distributions (for training): 
  p(x|y) 
Loss function: 
  mean(-(log p(x|y))) 
Optimizer: 
  Adam (
  Parameter Group 0
      amsgrad: False
      betas: (0.9, 0.999)
      eps: 1e-08
      lr: 0.001
      weight_decay: 0
  )


In [10]:
def train(epoch):
    train_loss = 0
    for x, y in tqdm(train_loader):
        x = x.to(device)
        y = torch.eye(10)[y].to(device)        
        loss = model.train({"x": x, "y": y})
        train_loss += loss
 
    train_loss = train_loss * train_loader.batch_size / len(train_loader.dataset)
    print('Epoch: {} Train loss: {:.4f}'.format(epoch, train_loss))
    return train_loss

In [11]:
def test(epoch):
    test_loss = 0
    for x, y in test_loader:
        x = x.to(device)
        y = torch.eye(10)[y].to(device)
        loss = model.test({"x": x, "y": y})
        test_loss += loss

    test_loss = test_loss * test_loader.batch_size / len(test_loader.dataset)
    print('Test loss: {:.4f}'.format(test_loss))
    return test_loss

In [12]:
def plot_reconstrunction(x, y):
    with torch.no_grad():
        z = p.forward(x, y, compute_jacobian=False)
        recon_batch = p.inverse(z, y).view(-1, 1, 28, 28)
    
        recon = torch.cat([x.view(-1, 1, 28, 28), recon_batch]).cpu()
        return recon
    
def plot_image_from_latent(z, y):
    with torch.no_grad():
        sample = p.inverse(z, y).view(-1, 1, 28, 28).cpu()
        return sample
    
def plot_reconstrunction_changing_y(x, y):
    y_change = torch.eye(10)[range(7)].to(device)
    batch_dummy = torch.ones(x.size(0))[:, None].to(device)    
    recon_all = []
    
    with torch.no_grad():
        for _y in y_change:
            z = p.forward(x, y, compute_jacobian=False)
            recon_batch = p.inverse(z, batch_dummy * _y[None,:]).view(-1, 1, 28, 28)
            recon_all.append(recon_batch)
    
        recon_changing_y = torch.cat(recon_all)
        recon_changing_y = torch.cat([x.view(-1, 1, 28, 28), recon_changing_y]).cpu()
        return recon_changing_y

In [13]:
writer = SummaryWriter()

plot_number = 5

z_sample = 0.5 * torch.randn(64, z_dim).to(device)
y_sample = torch.eye(10)[[plot_number]*64].to(device)

_x, _y = iter(test_loader).next()
_x = _x.to(device)
_y = torch.eye(10)[_y].to(device)

for epoch in range(1, epochs + 1):
    train_loss = train(epoch)
    test_loss = test(epoch)
    
    recon = plot_reconstrunction(_x[:8], _y[:8])
    sample = plot_image_from_latent(z_sample, y_sample)
    recon_changing_y = plot_reconstrunction_changing_y(_x[:8], _y[:8])

    writer.add_scalar('train_loss', train_loss.item(), epoch)
    writer.add_scalar('test_loss', test_loss.item(), epoch)      
    
    writer.add_images('Image_from_latent', sample, epoch)
    writer.add_images('Image_reconstrunction', recon, epoch)
    writer.add_images('Image_reconstrunction_change_y', recon_changing_y, epoch)
    
writer.close()

100%|██████████| 469/469 [00:12<00:00, 37.94it/s]

Epoch: 1 Train loss: 1440.0148



  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: 1833.4519


100%|██████████| 469/469 [00:12<00:00, 37.55it/s]


...

Epoch: 99 Train loss: 945.7551



  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: 1527.3599


100%|██████████| 469/469 [00:12<00:00, 38.51it/s]


Epoch: 100 Train loss: 945.5173
Test loss: 1348.1760
